In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key=os.getenv('GOOGLE_API_KEY')
if not api_key:
    raise ('Not key')

In [3]:
MAX_PAGE=2
URL='https://cafef.vn/tai-chinh-ngan-hang.chn'
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
    'Accept-Language': 'vi-VN,vi;q=0.9,en-US;q=0.8,en;q=0.7'
}
response=requests.get(URL,headers=HEADERS,timeout=10)

In [4]:
if response.status_code == 200:
    soup=BeautifulSoup(response.content,'html.parser')
# soup

In [5]:
hiden_input=soup.find('input',{'id':'hdZoneId'})
hiden_input

<input id="hdZoneId" name="hdZoneId" type="hidden" value="18834">
<input id="hdZoneUrl" name="hdZoneUrl" type="hidden" value="tai-chinh-ngan-hang"/>
<input id="hdParentUrl" name="hdParentUrl" type="hidden" value=""/>
<input id="hdZoneParentUrl" name="hdZoneParentUrl" type="hidden" value=""/>
</input>

In [6]:
zoneId=hiden_input.get('value')
zoneId

'18834'

In [7]:
all_link=[]
base_api='https://cafef.vn/timelinelist/{}/{}.chn'
for page in range(1, MAX_PAGE+1):
    api_url=base_api.format(zoneId,page)

    r=requests.get(api_url,headers=HEADERS,timeout=10)
    if r.status_code==200:
        api_soup=BeautifulSoup(r.content,'html.parser')
        links_found = 0

        for a_tag in api_soup.find_all('a',href=True):
            href=a_tag['href']
            if href.endswith('.chn') and len(href)>25:
                full_link = "https://cafef.vn" + href if not href.startswith('http') else href
                if full_link not in all_link:
                    all_link.append(full_link)
time.sleep(1)
print(f'{len(all_link)}')
print(f'{all_link[:5]}')

27
['https://cafef.vn/nong-vang-mieng-chinh-thuc-thiet-lap-moc-gia-chua-ai-tung-nghi-toi-18826012614045161.chn', 'https://cafef.vn/sang-26-1-vang-the-gioi-vuot-moc-5000-usd-ounce-gia-vang-trong-nuoc-lap-tuc-tang-vot-188260126090746154.chn', 'https://cafef.vn/cap-nhat-thi-truong-tien-te-ty-gia-usd-tu-do-tang-manh-nhnn-hut-rong-hon-40000-ty-dong-trong-tuan-qua-188260126101356933.chn', 'https://cafef.vn/cu-tri-kien-nghi-co-chinh-sach-binh-on-gia-vang-188260126092416542.chn', 'https://cafef.vn/tu-duy-tai-chinh-cua-nguoi-tre-hien-dai-dung-doi-giau-moi-tich-luy-hay-bat-dau-tu-tu-quy-thinh-vuong-188260124100851737.chn']


In [8]:
final_data=[]
for i,url in enumerate(all_link[:5]):
    r=requests.get(url,headers=HEADERS,timeout=10)
    soup=BeautifulSoup(r.content,'html.parser')

    title=soup.find('h1',class_='title')
    title_text=title.text.strip() if title else''

    content_div=soup.find('div',class_='detail-content')
    if not content_div:
        content_div=soup.find('div',class_='contentdetail')
    content_text=''
    if content_div:
        for script in content_div(["script", "style", "iframe"]):
            script.decompose()
        paragraphs = [p.text.strip() for p in content_div.find_all('p') if p.text.strip()]
        content_text = "\n".join(paragraphs)
    if title_text and len(content_text) > 50:
        final_data.append({
            'url':url,
            'title':title_text,
            'content':content_text
        })
    time.sleep(1)
df = pd.DataFrame(final_data)

In [9]:
df.head(5)

,url,title,content
0,https://cafef.vn/nong-vang-mieng-chinh-thuc-th...,NÓNG: Vàng miếng chính thức thiết lập mốc giá ...,Thị trường vàng trong nước và quốc tế đang trả...
1,https://cafef.vn/sang-26-1-vang-the-gioi-vuot-...,"Giá vàng SJC, giá vàng nhẫn đồng loạt tăng mạn...","Cập nhật đến 9h20, các thương hiệu lớn đều đã ..."
2,https://cafef.vn/cap-nhat-thi-truong-tien-te-t...,Cập nhật thị trường tiền tệ: Tỷ giá USD tự do ...,Thị trường ngoại tệ: Trong tuần giao dịch từ 1...
3,https://cafef.vn/cu-tri-kien-nghi-co-chinh-sac...,Cử tri kiến nghị có chính sách bình ổn giá vàng,Cử tri tỉnh Lâm Đồng vừa kiến nghị Ngân hàng N...


In [10]:
import re

def clean_code(text):
    text=re.sub(r'[^\w\s.,%()-]',' ',text)
    text=re.sub(r'(Theo|Nguồn):.*$',' ',text)
    text=re.sub(r'\s+',' ',text).strip()
    return text

df['content']=df['content'].apply(clean_code)
df['content'].iloc[0]

'Thị trường vàng trong nước và quốc tế đang trải qua những ngày biến động dữ dội nhất trong lịch sử. Sáng nay, ngày 26 01 2026, giá vàng trong nước đã chính thức thiết lập những cột mốc giá chưa từng có, phản ánh sức nóng từ thị trường kim loại quý toàn cầu. Cụ thể, tại Bảo Tín Mạnh Hải hôm nay So với phiên giao dịch ngày hôm qua (25 01), giá vàng tại Bảo Tín Mạnh Hải đã ghi nhận mức tăng mạnh mẽ, dao động từ 2,1 đến 2,2 triệu đồng lượng ở cả hai chiều mua và bán. Nhìn lại cả tuần qua, giá vàng đã có một cuộc viễn chinh không tưởng. Từ mức quanh ngưỡng 163 triệu đồng lượng vào đầu tuần, giá vàng đã bứt phá thêm khoảng 13,5 triệu đồng lượng . Như vậy, chỉ trong vòng 7 ngày, những nhà đầu tư nắm giữ vàng đã thu về khoản lợi nhuận khổng lồ, đưa giá trị tài sản bằng vàng lên một tầm cao mới. Sự bùng nổ của giá vàng trong nước hoàn toàn nằm trong quỹ đạo của giá vàng thế giới. Sáng nay, giá vàng giao ngay trên sàn Kitco đã chính thức xuyên thủng ngưỡng kháng cự tâm lý cực mạnh 5.000 USD oun

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

split=RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=100,
    separators=['\n\n','\n','.',' ','']
)
    
chunk=[]
for index,row in df.iterrows():
    doc = Document(
        page_content=row['content'],
        metadata={
            'source':row['url'],
            'title':row['title']
        }
    )
    text_slpit=split.split_documents([doc])
    chunk.extend(text_slpit)
# chunk

2026-01-26 19:26:24.408479: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-26 19:26:24.584681: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-26 19:26:26.336021: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/chinghia/miniconda3/lib/python3.13/site-packages/keras/src/export/tf2onnx_lib.py:8: F

In [12]:
from langchain_huggingface import HuggingFaceEmbeddings

model='BAAI/bge-m3'
# model='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
embeddings=HuggingFaceEmbeddings(model_name=model)

In [13]:
from langchain_chroma.vectorstores import Chroma

chroma=Chroma.from_documents(
    chunk,
    embeddings,
    persist_directory='./chroma_db_economy'
)

retriever=chroma.as_retriever(search_kwargs={'k':50})

In [14]:
y='lãi suất tiền gửi tại PGBank'

context=retriever.invoke(y)

for doc in context:
    print(f'{doc.page_content}')
print(f'-------')
# context='\n\n'.join([doc.page_content for doc in context])
# context

. Với nhóm kỳ hạn 6 - 13 tháng, lãi suất tiền gửi tại PGBank vẫn ở mức cao. Cụ thể, kỳ hạn 6 9 tháng được niêm yết 7,1% năm, còn kỳ hạn 12 13 tháng giữ nguyên 7,2% năm. Sau điều chỉnh, đây trở thành mức lãi suất cao nhất trong biểu lãi suất lĩnh lãi cuối kỳ của ngân hàng này đồng thời lãi suất kỳ hạn 6 tháng còn cao hơn cả 36 tháng. Ở biểu lãi suất tiết kiệm trả lãi hằng tháng, PGBank cũng điều chỉnh giảm lãi suất các kỳ hạn dài
. Với nhóm kỳ hạn 6 - 13 tháng, lãi suất tiền gửi tại PGBank vẫn ở mức cao. Cụ thể, kỳ hạn 6 9 tháng được niêm yết 7,1% năm, còn kỳ hạn 12 13 tháng giữ nguyên 7,2% năm. Sau điều chỉnh, đây trở thành mức lãi suất cao nhất trong biểu lãi suất lĩnh lãi cuối kỳ của ngân hàng này đồng thời lãi suất kỳ hạn 6 tháng còn cao hơn cả 36 tháng. Ở biểu lãi suất tiết kiệm trả lãi hằng tháng, PGBank cũng điều chỉnh giảm lãi suất các kỳ hạn dài
. Với nhóm kỳ hạn 6 - 13 tháng, lãi suất tiền gửi tại PGBank vẫn ở mức cao. Cụ thể, kỳ hạn 6 9 tháng được niêm yết 7,1% năm, còn kỳ hạ

In [15]:
# from langchain_chroma.vectorstores import Chroma

# collection_metadata={'hnsw:space':'l2'}

# chroma_dis=Chroma.from_documents(
#     chunk,
#     embeddings,
#     persist_directory='./chroma_db_energy',
#     collection_metadata=collection_metadata
# )

In [16]:
# y_dis='lãi suất tiền gửi tại PGBank'
# retriever_dis=chroma_dis.as_retriever(search_kwargs={'k':10})
# context_dis=retriever_dis.invoke(y_dis)
# for doc in context_dis:
#     print(f'{doc.page_content}')
# # context_dis='\n\n'.join([doc.page_content for doc in context_dis])
# # context_dis

In [17]:
# from sentence_transformers import CrossEncoder

# model_cross='BAAI/bge-reranker-v2-m3'

# rerank=CrossEncoder(
#     model_cross,
#     max_length=512,
#     device='cpu'
# )
# pairs=[[y,doc.page_content]for doc in context]
# score=rerank.predict(pairs)

# pairs_dis=[[y_dis,doc.page_content] for doc in context_dis]
# score_dis=rerank.predict(pairs_dis)

In [18]:
# print(f'score {score}')
# print(f'-------')
# print(f'score_dis {score_dis}')

In [19]:
# for i in score:
#     energy=-float(i)
#     print(f'energy {energy}')
# print(f'-------')

# for i_dis in score_dis:
#     energy_dis=-float(i_dis)
#     print(f'energy_dis {energy_dis}')

In [20]:
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_google_genai import ChatGoogleGenerativeAI

# model='gemini-2.5-flash'
# llm=ChatGoogleGenerativeAI(model=model,temperature=0)
# def answer(query):
#     context=retriever.invoke(query)
#     context='\n\n'.join([doc.page_content for doc in context])

#     template='''
#     `context{context}

#     question{question}
#     '''
#     prompt=ChatPromptTemplate.from_template(template)

#     chain= prompt | llm | StrOutputParser()

#     return chain.invoke({'context':context, 'question':query})

In [21]:
# y='Gia co phieu VCB'

# a=answer(y)

# a
# #

In [28]:
import numpy as np
from scipy.spatial.distance import cdist



def energy_distance(X,Y):
    X = np.asarray(X)
    Y = np.asarray(Y)
    N=X.shape[0]
    M=Y.shape[0]

    d_xy=cdist(X,Y,metric='euclidean')
    E_xy=np.mean(d_xy)

    d_xx=cdist(X,X,metric='euclidean')
    E_xx=np.mean(d_xx)

    d_yy=cdist(Y,Y,metric='euclidean')
    E_yy=np.mean(d_yy)

    ED= 2*E_xy - E_xx - E_yy

    return max(0, ED)


In [23]:
X = [[1,1],[2,3]]
Y = [[1,1],[2,3]]

ed=energy_distance(X,Y)
print(f'{ed}')

0.0


In [24]:
context_list=[doc.page_content for doc in context]

context_list

['. Với nhóm kỳ hạn 6 - 13 tháng, lãi suất tiền gửi tại PGBank vẫn ở mức cao. Cụ thể, kỳ hạn 6 9 tháng được niêm yết 7,1% năm, còn kỳ hạn 12 13 tháng giữ nguyên 7,2% năm. Sau điều chỉnh, đây trở thành mức lãi suất cao nhất trong biểu lãi suất lĩnh lãi cuối kỳ của ngân hàng này đồng thời lãi suất kỳ hạn 6 tháng còn cao hơn cả 36 tháng. Ở biểu lãi suất tiết kiệm trả lãi hằng tháng, PGBank cũng điều chỉnh giảm lãi suất các kỳ hạn dài',
 '. Với nhóm kỳ hạn 6 - 13 tháng, lãi suất tiền gửi tại PGBank vẫn ở mức cao. Cụ thể, kỳ hạn 6 9 tháng được niêm yết 7,1% năm, còn kỳ hạn 12 13 tháng giữ nguyên 7,2% năm. Sau điều chỉnh, đây trở thành mức lãi suất cao nhất trong biểu lãi suất lĩnh lãi cuối kỳ của ngân hàng này đồng thời lãi suất kỳ hạn 6 tháng còn cao hơn cả 36 tháng. Ở biểu lãi suất tiết kiệm trả lãi hằng tháng, PGBank cũng điều chỉnh giảm lãi suất các kỳ hạn dài',
 '. Với nhóm kỳ hạn 6 - 13 tháng, lãi suất tiền gửi tại PGBank vẫn ở mức cao. Cụ thể, kỳ hạn 6 9 tháng được niêm yết 7,1% năm,

In [59]:
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
def retrieve_with_energy_clustering (query,base_retriever , embeddings, k_cluters=5):
    raw_doc=base_retriever.invoke(query)

    doctexts=[doc.page_content for doc in raw_doc]

    doc_vector=np.array(embeddings.embed_documents(doctexts))
    query_vector=np.array(embeddings.embed_query(query)).reshape(1,-1)

    sims = cosine_similarity(query_vector, doc_vector)[0]
    print("Max cosine:", np.max(sims))
    if np.max(sims) < 0.35:
        return [] 

    actual_k=min(k_cluters,len(doc_vector))
    kmeans=KMeans(n_clusters=actual_k,random_state=42,n_init='auto')
    labels=kmeans.fit_predict(doc_vector)

    best_energy = float('inf')
    best_cluster_idx = -1


    for i in range(actual_k):
        indices = np.where(labels==i)[0]
        cluster_vectors=doc_vector[indices]

        energy=energy_distance(query_vector,cluster_vectors)

        if best_energy>energy:
            best_energy=energy
            best_cluster_idx = i

    winning_indices= np.where(labels==best_cluster_idx)[0]
    final_docs=[raw_doc[i] for i in winning_indices]
    print(f'{best_energy}')
    return final_docs

In [61]:
query='Gia vang cao nhat bao nhieu'

final_results=retrieve_with_energy_clustering(query,retriever,embeddings)

Max cosine: 0.4656088092606721
1.5355666074873346


In [62]:
for doc in final_results[:3]:
    print(doc.page_content)

. Trước đó, giá vàng tăng mạnh 64% trong năm 2025, được hỗ trợ bởi nhu cầu trú ẩn an toàn duy trì ở mức cao, chính sách tiền tệ nới lỏng của Mỹ, lực mua bền bỉ từ các ngân hàng trung ương trong đó Trung Quốc kéo dài chuỗi mua vàng sang tháng thứ 14 liên tiếp trong tháng 12 cùng với dòng vốn kỷ lục đổ vào các quỹ hoán đổi danh mục (ETF). Chúng tôi kỳ vọng giá vàng sẽ còn dư địa tăng tiếp. Dự báo hiện tại cho thấy giá có thể đạt đỉnh quanh mốc 5
. Trước đó, giá vàng tăng mạnh 64% trong năm 2025, được hỗ trợ bởi nhu cầu trú ẩn an toàn duy trì ở mức cao, chính sách tiền tệ nới lỏng của Mỹ, lực mua bền bỉ từ các ngân hàng trung ương trong đó Trung Quốc kéo dài chuỗi mua vàng sang tháng thứ 14 liên tiếp trong tháng 12 cùng với dòng vốn kỷ lục đổ vào các quỹ hoán đổi danh mục (ETF). Chúng tôi kỳ vọng giá vàng sẽ còn dư địa tăng tiếp. Dự báo hiện tại cho thấy giá có thể đạt đỉnh quanh mốc 5
. Trước đó, giá vàng tăng mạnh 64% trong năm 2025, được hỗ trợ bởi nhu cầu trú ẩn an toàn duy trì ở mức 